In [1]:
import json
import os
import csv
import math
import numpy as np
import pandas as pd

from automates.model_assembly.networks import GroundedFunctionNetwork
from automates.model_analysis.sensitivity import (
    SensitivityAnalyzer,
    SADependentVariable,
)

In this first section, variable bounds, fixed inputs, and outputs are defined.

In [2]:
def gather_stemp_soilt_inputs():
    bounds = {
        "stemp_soilt::stemp_soilt.soilt::srad::-1": [0.5, 30],
        "stemp_soilt::stemp_soilt.soilt::tmax::-1": [0, 40],
        # "stemp_soilt::stemp_soilt.soilt::nlayr::-1": [1, 20],
    }

    inputs = {
        # "stemp_soilt::stemp_soilt.soilt::nlayr::-1": SADependentVariable(
        #     "stemp_soilt::stemp_soilt.soilt::nlayr::-1",
        #     ["stemp_soilt::stemp_soilt.soilt::nlayr::-1"],
        #     lambda nlayr: int(nlayr),
        # ),
        "stemp_soilt::stemp_soilt.soilt::nlayr::-1": 10,
        "stemp_soilt::stemp_soilt.soilt::atot::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::tma::-1": [0, 0, 0, 0, 0],
        "stemp_soilt::stemp_soilt.soilt::ww::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::albedo::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::b::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::cumdpt::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::doy::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::dp::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::hday::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::pesw::-1": 1,
        "stemp_soilt::stemp_soilt.soilt::tamp::-1": 0,
        "stemp_soilt::stemp_soilt.soilt::tav::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::tav::-1",
            ["stemp_soilt::stemp_soilt.soilt::tmax::-1"],
            lambda tmax: tmax - 5,
        ),
        "stemp_soilt::stemp_soilt.soilt::tavg::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::tavg::-1",
            [
                "stemp_soilt::stemp_soilt.soilt::tmax::-1",
                "stemp_soilt::stemp_soilt.soilt::tav::-1",
            ],
            lambda tmax, tav: (tmax + tav) / 2,
        ),
        "stemp_soilt::stemp_soilt.soilt::dsmid::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::dsmid::-1",
            ["stemp_soilt::stemp_soilt.soilt::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
        "stemp_soilt::stemp_soilt.soilt::st::-1": SADependentVariable(
            "stemp_soilt::stemp_soilt.soilt::st::-1",
            ["stemp_soilt::stemp_soilt.soilt::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
    }

    output_base_names = ["atot", "tma", "srftemp", "st"]
    # output_base_names = ["tma"]

    return (bounds, inputs, output_base_names)


def gather_stemp_epic_soilt_inputs():
    bounds = {
        # "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::srad::-1": [0.5, 30],
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmax::-1": [0, 40],
        # "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1": [1, 20],
    }

    inputs = {
        # "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1": SADependentVariable(
        #     "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1",
        #     ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1"],
        #     lambda nlayr: int(nlayr),
        # ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::b::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::bcv::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1": 10,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tma::-1": [0, 0, 0, 0, 0],
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::ww::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::b::-1": 0,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::cumdpt::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dp::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::pesw::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::wetday::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::wft::-1": 20,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::ww::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tav::-1": 1,
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmin::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmin::-1",
            ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmax::-1"],
            lambda tmax: tmax - 5,
        ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tavg::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tavg::-1",
            [
                "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tmax::-1",
                "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::tav::-1",
            ],
            lambda tmax, tav: (tmax + tav) / 2,
        ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dsmid::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dsmid::-1",
            ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
        "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::st::-1": SADependentVariable(
            "stemp_epic_soilt::stemp_epic_soilt.soilt_epic::st::-1",
            ["stemp_epic_soilt::stemp_epic_soilt.soilt_epic::nlayr::-1"],
            lambda nlayr: [0] * nlayr,
        ),
    }

    output_base_names = ["tma", "srftemp", "st", "x2_avg"]

    return (bounds, inputs, output_base_names)

Next, the location of the GrFN files and $N$, the number of samples to analyze when generating Si, and the option to compute second-order indices.

In [3]:
N = 100
compute_S2 = False
file = "stemp_soilt--GrFN.json"  # file location
file_epic = "stemp_epic_soilt--GrFN.json"  # epic file location

The sensitivity analysis follows.

In [4]:
if file is not None:
    print("Reading soilt json file...")
    if not os.path.exists(file):
        raise Exception(f"Error: File '{file}' does not exist")
    (bounds, inputs, outputs) = gather_stemp_soilt_inputs()
    
    print("Parsing grfn json...")
    grfn = None
    try:
        grfn = GroundedFunctionNetwork.from_json(file)
    except Exception:
        raise Exception(f"Error: Unable to process grfn json file {file}")
        
    print("Running sensitivity analysis...")
    analyzer = SensitivityAnalyzer()
    Si_list = analyzer.Si_from_Sobol(N, grfn, bounds, inputs, outputs, calc_2nd=compute_S2)
    model_inputs = [
        {
            "variable_name": k,
            "bounds": v,
        }
        for k, v in bounds.items()
    ]
    model_outputs = [{"variable_name": n} for n in outputs]
    results = {
        "grfn_uid": grfn.uid,
        "input_variables": model_inputs,
        "output_variables": model_outputs,
        "sensitivity_indices": [Si.to_dict() for Si in Si_list],
    }

Reading soilt json file...
Parsing grfn json...
Running sensitivity analysis...
{'stemp_soilt::stemp_soilt.soilt::albedo::-1': 0,
 'stemp_soilt::stemp_soilt.soilt::atot::-1': 0,
 'stemp_soilt::stemp_soilt.soilt::b::-1': 0,
 'stemp_soilt::stemp_soilt.soilt::cumdpt::-1': 1,
 'stemp_soilt::stemp_soilt.soilt::doy::-1': 0,
 'stemp_soilt::stemp_soilt.soilt::dp::-1': 1,
 'stemp_soilt::stemp_soilt.soilt::dsmid::-1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'stemp_soilt::stemp_soilt.soilt::hday::-1': 0,
 'stemp_soilt::stemp_soilt.soilt::nlayr::-1': 10,
 'stemp_soilt::stemp_soilt.soilt::pesw::-1': 1,
 'stemp_soilt::stemp_soilt.soilt::srad::-1': array([ 6.98193359, 15.79736328,  6.98193359, 15.79736328, 21.73193359,
        1.04736328, 21.73193359,  1.04736328, 29.10693359, 23.17236328,
       29.10693359, 23.17236328, 14.35693359,  8.42236328, 14.35693359,
        8.42236328, 10.66943359,  4.73486328, 10.66943359,  4.73486328,
       25.41943359, 19.48486328, 25.41943359, 19.48486328, 18.04443359,
     

{'atot': array([ 2.51140e+00,  3.08820e+00,  2.57145e+01,  2.62914e+01,
        2.33858e+01,  2.18103e+01,  6.58890e+00,  5.01350e+00,
        1.37033e+01,  1.34516e+01,  3.69065e+01,  3.66547e+01,
        3.30079e+01,  3.26238e+01,  1.62110e+01,  1.58270e+01,
        1.77816e+01,  1.73094e+01,  1.09847e+01,  1.05125e+01,
        3.85503e+01,  3.82786e+01,  3.17535e+01,  3.14817e+01,
        8.20660e+00,  7.87400e+00,  1.40970e+00,  1.07720e+00,
        2.71531e+01,  2.86113e+01,  2.03563e+01,  2.18145e+01,
        5.78420e+00,  5.72480e+00,  1.64873e+01,  1.64280e+01,
        2.69090e+01,  2.68877e+01,  3.76121e+01,  3.75909e+01,
        1.72604e+01,  1.64408e+01,  7.96350e+00,  7.14400e+00,
        3.64691e+01,  3.72427e+01,  2.71722e+01,  2.79458e+01,
        1.17043e+01,  1.20733e+01,  2.24075e+01,  2.27764e+01,
        3.24160e+01,  3.11471e+01,  3.11920e+00,  1.85020e+00,
        2.09250e+00,  2.39950e+00,  3.27956e+01,  3.31027e+01,
        2.11831e+01,  2.16803e+01,  1.18863e+0

/Users/awinchester/.pyenv/versions/causaleffect/lib/python3.8/site-packages/SALib/analyze/sobol.py:93: RuntimeWarning: invalid value encountered in true_divide
  Y = (Y - Y.mean()) / Y.std()


In [5]:
if file_epic is not None:
    print("Reading soilt epic json file...")
    if not os.path.exists(file_epic):
        raise Exception(f"Error: File '{file_epic}' does not exist")
    (bounds_epic, inputs_epic, outputs_epic) = gather_stemp_epic_soilt_inputs()
    
    print("Parsing grfn_epic json...")
    grfn_epic = None
    try:
        grfn_epic = GroundedFunctionNetwork.from_json(file_epic)
    except Exception:
        raise Exception(f"Error: Unable to process grfn_epic json file {file_epic}")
        
    print("Running epic sensitivity analysis...")
    analyzer = SensitivityAnalyzer()
    Si_list_epic = analyzer.Si_from_Sobol(N, grfn_epic, bounds_epic, inputs_epic, outputs_epic, calc_2nd=compute_S2)
    model_inputs_epic = [
        {
            "variable_name": k,
            "bounds": v,
        }
        for k, v in bounds_epic.items()
    ]
    model_outputs_epic = [{"variable_name": n} for n in outputs_epic]
    results_epic = {
        "grfn_uid": grfn_epic.uid,
        "input_variables": model_inputs_epic,
        "output_variables": model_outputs_epic,
        "sensitivity_indices": [Si.to_dict() for Si in Si_list_epic],
    }

Reading soilt epic json file...
Parsing grfn_epic json...
Running epic sensitivity analysis...
{'stemp_epic_soilt::stemp_epic_soilt.soilt_epic::b::-1': 0,
 'stemp_epic_soilt::stemp_epic_soilt.soilt_epic::bcv::-1': 1,
 'stemp_epic_soilt::stemp_epic_soilt.soilt_epic::cumdpt::-1': 1,
 'stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dp::-1': 1,
 'stemp_epic_soilt::stemp_epic_soilt.soilt_epic::dsmid::-1': [0,
                                                              0,
                                                              0,
                                                              0,
                                                              0,
                                                              0,
                                                              0,
                                                              0,
                                                              0,
                                                              0],
 'stem

{'srftemp': array([ -1. ,  -1. ,  -1. , -61.6, -43.6, -43.6, -97.6,  -7.6,  -7.6,
       -25.6, -79.6, -79.6,  -7.6, -25.6, -25.6, -79.6, -97.6, -97.6,
       -43.6,  -1. ,  -1. ,  -1. , -61.6, -61.6,  -1. ,  -1. ,  -1. ,
       -38.8, -57.2, -57.2, -74.8, -21.2, -21.2,  -2.8, -93.2, -93.2,
       -20.8,  -3.2,  -3.2, -92.8, -75.2, -75.2, -56.8,  -1. ,  -1. ,
        -1. , -39.2, -39.2,  -1. , -12.4, -12.4, -48. , -84.4, -84.4,
       -84. ,  -1. ,  -1. , -12. , -48.4, -48.4,  -1. ,  -1. ,  -1. ,
       -66. , -30.4, -30.4, -30. ,  -1. ,  -1. ,  -1. , -66.4, -66.4,
        -1. , -12. , -12. , -30. , -84. , -84. , -66. ,  -1. ,  -1. ,
        -1. , -48. , -48. , -12. ,  -1. ,  -1. , -84. , -30. , -30. ,
       -48. ,  -1. ,  -1. ,  -1. , -66. , -66. ,  -1. ,  -1. ,  -1. ,
       -57.2, -57.2, -57.2, -93.2, -21.2, -21.2, -21.2, -93.2, -93.2,
        -3.2,  -3.2,  -3.2, -75.2, -75.2, -75.2, -39.2,  -1. ,  -1. ,
        -1. , -39.2, -39.2,  -1. ,  -1. ,  -1. , -43.6, -43.6, -43.6,
       -

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[  25.   70.   70. -154. -109. -109. -244.  -19.  -19.  -64. -199. -199.
  -19.  -64. 

The results are displayed.

In [6]:
def to_table(csv_table): 
    old_table = np.array(csv_table)
    col_names = old_table[0,:]
    row_names = old_table[:,0]

    new_table = old_table[1:len(row_names), 1:len(col_names)]
    col_names = np.delete(col_names, 0)
    row_names = np.delete(row_names, 0)
    
    return pd.DataFrame(new_table, columns=col_names, index=row_names)

def pd_tables(sa_obj, compute_S2):
#     json_file = file

#     sa_obj = {}
#     with open(json_file) as f:
#         sa_obj = json.load(f)

    outputs = []
    inputs = []

#     for v in sa_obj["output_variables"]:
#         outputs.append(v["variable_name"])
        
    for i in range(len(sa_obj["sensitivity_indices"])):
        outputs.append(f"v{i+1}")

    for v in sa_obj["input_variables"]:
        identifier = v["variable_name"]
        name = identifier.split("::")[-2]
        inputs.append(name)

    table = [[""] * (len(inputs) + 1) for v in range(len(outputs) + 1)]

    for col in range(len(inputs)):
        table[0][col + 1] = inputs[col]

    for idx,_ in enumerate(outputs):
        si = sa_obj['sensitivity_indices'][idx]
        table[idx + 1][0] = outputs[idx]
        for i in range(len(si['S1'])):
            S1_val = "--" if si['S1'][i] is None or math.isnan(si['S1'][i]) else round(si['S1'][i], 3)
            S1_conf = "--" if si['S1_conf'][i] is None or math.isnan(si['S1_conf'][i]) else round(si['S1_conf'][i], 3)

            table[idx + 1][i + 1] = f"{S1_val} +/- {S1_conf}"
            
    # ST Table
    st_table = [[""] * (len(inputs) + 1) for v in range(len(outputs) + 1)]

    for col in range(len(inputs)):
        st_table[0][col + 1] = inputs[col]

    for idx,_ in enumerate(outputs):
        si = sa_obj['sensitivity_indices'][idx]
        st_table[idx + 1][0] = outputs[idx]
        for i in range(len(si['ST'])):
            ST_val = "--" if si['ST'][i] is None or math.isnan(si['ST'][i]) else round(si['ST'][i], 3)
            ST_conf = "--" if si['ST_conf'][i] is None or math.isnan(si['ST_conf'][i]) else round(si['ST_conf'][i], 3)

            st_table[idx + 1][i + 1] = f"{ST_val} +/- {ST_conf}"

    # S2 Table
    if compute_S2:
        n_inputs = len(inputs)
        n_cols = math.comb(n_inputs, 2)
        s2_table = [[""] * (n_cols + 1) for v in range(len(outputs) + 1)]
        s2_columns = [""]
        for i in range(len(inputs)):
            for j in range(i+1, n_inputs):
                s2_columns.append(f"{inputs[i]}, {inputs[j]}")
        s2_table[0] =  s2_columns

        for idx,_ in enumerate(outputs):
            si = sa_obj['sensitivity_indices'][idx]
            s2_raw = si['S2']
            s2_raw_conf = si['S2_conf']
            s2_trimmed = [outputs[idx]]
            for i in range(n_inputs):
                for j in range(i+1, n_inputs):
                    s2_val = "--" if s2_raw[i][j] is None or math.isnan(s2_raw[i][j]) else round(s2_raw[i][j], 3)
                    s2_conf = "--" if s2_raw_conf[i][j] is None or math.isnan(s2_raw_conf[i][j]) else round(s2_raw_conf[i][j], 3)
                    s2_trimmed.append(f"{s2_val} +/- {s2_conf}")
            s2_table[idx + 1] = s2_trimmed
        return (to_table(table), to_table(s2_table), to_table(st_table))
    return (to_table(table), to_table(st_table))
            



#     csv_file_name = f"{json_file.rsplit('.', 1)[0]}.csv"
#     with open(csv_file_name, "w") as csv_file:
#         csvWriter = csv.writer(csv_file, delimiter=',')
#         csvWriter.writerows(table)







In [7]:
if file is not None:
#     out_file = f"soilt_model-{N}--SA.json"
#     print(f"Writing results to {out_file}")
#     json.dump(results, open(out_file, "w"))
#     csv_result = to_csv(results)
    if compute_S2:
        (s1, s2, st) = pd_tables(results, compute_S2)
    else:
        (s1, st) = pd_tables(results, compute_S2)
if file_epic is not None:
#     out_file_epic = f"soilt_epic_model-{N}--SA.json"
#     print(f"Writing results to {out_file_epic}")
#     json.dump(results, open(out_file_epic, "w"))
#     csv_epic_result = to_csv(results_epic)
    if compute_S2:
        (s1_epic, s2_epic, st_epic) = pd_tables(results_epic, compute_S2)
    else:
        (s1_epic, st_epic) = pd_tables(results_epic, compute_S2)

In [8]:
s1

,srad,tmax
v1,0.006 +/- 0.013,1.02 +/- 0.227
v2,0.006 +/- 0.013,1.02 +/- 0.228
v3,-- +/- --,-- +/- --
v4,-- +/- --,-- +/- --
v5,-- +/- --,-- +/- --
v6,-- +/- --,-- +/- --
v7,0.006 +/- 0.014,1.02 +/- 0.255
v8,-- +/- --,-- +/- --
v9,-- +/- --,-- +/- --
v10,-- +/- --,-- +/- --


In [9]:
s1_epic

,tmax
v1,0.964 +/- 0.216
v2,0.964 +/- 0.22
v3,-- +/- --
v4,-- +/- --
v5,-- +/- --
v6,0.964 +/- 0.211
v7,0.964 +/- 0.205
v8,-- +/- --
v9,-- +/- --
v10,-- +/- --


In [10]:
st

,srad,tmax
v1,0.002 +/- 0.001,1.02 +/- 0.183
v2,0.002 +/- 0.001,1.02 +/- 0.193
v3,-- +/- --,-- +/- --
v4,-- +/- --,-- +/- --
v5,-- +/- --,-- +/- --
v6,-- +/- --,-- +/- --
v7,0.002 +/- 0.001,1.02 +/- 0.204
v8,-- +/- --,-- +/- --
v9,-- +/- --,-- +/- --
v10,-- +/- --,-- +/- --


In [11]:
st_epic

,tmax
v1,0.966 +/- 0.195
v2,0.966 +/- 0.187
v3,-- +/- --
v4,-- +/- --
v5,-- +/- --
v6,0.966 +/- 0.195
v7,0.966 +/- 0.179
v8,-- +/- --
v9,-- +/- --
v10,-- +/- --


In [12]:
s2

NameError: name 's2' is not defined

In [ ]:
s2_epic